This notebook shows how to fine-tune Mistral 7B on a sample of ultrachat with a CPU. You don't need CUDA or a GPU to run it but you will need a lot a CPU RAM (at least 33 GB)

intel-extension-for-transformers doesn't support the last version of PEFT (as of December 2023). Install the version 0.6.2 if the last version still doesn't work when you run this notebook:

In [ ]:
!pip install bitsandbytes accelerate trl peft==0.6.2 datasets
!pip install intel-extension-for-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 66.6 MB/s eta 0:00:00


Import all the necessary packages.

In [ ]:
import torch
from intel_extension_for_transformers.transformers.modeling import AutoModelForCausalLM
from datasets import load_dataset
from peft import get_peft_model,  LoraConfig, TaskType, prepare_model_for_kbit_training
from transformers import (
    AutoTokenizer,
    TrainingArguments
)

from trl import SFTTrainer

/usr/local/lib/python3.10/dist-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


Load the tokenizer and configure padding

In [ ]:
model_name = "mistralai/Mistral-7B-v0.1"
#Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, add_eos_token=True, use_fast=True)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id =  tokenizer.unk_token_id
tokenizer.padding_side = 'left'

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

Load and preprocess the version of ultrachat prepared by Hugging Face.
Since each row is a full dialog that can be very long, I only kept the first two turns to reduce the sequence length of the training examples.

In [ ]:
def format_ultrachat(ds):
  text = []
  for row in ds:
    if len(row['messages']) > 2:
      text.append("### Human: "+row['messages'][0]['content']+"### Assistant: "+row['messages'][1]['content']+"### Human: "+row['messages'][2]['content']+"### Assistant: "+row['messages'][3]['content'])
    else: #not all tialogues have more than one turn
      text.append("### Human: "+row['messages'][0]['content']+"### Assistant: "+row['messages'][1]['content'])
  ds = ds.add_column(name="text", column=text)
  return ds
dataset_train_sft = load_dataset("HuggingFaceH4/ultrachat_200k", split="train_sft")
dataset_test_sft = load_dataset("HuggingFaceH4/ultrachat_200k", split="test_sft[:5%]")

dataset_test_sft = format_ultrachat(dataset_test_sft)
dataset_train_sft = format_ultrachat(dataset_train_sft)


Load the model and prepare it to be fine-tuned with QLoRA.

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
          model_name, load_in_4bit=True, use_llm_runtime=False, torch_dtype=torch.float32, low_cpu_mem_usage=True
)
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing = True)
#Configure the pad token in the model
model.config.pad_token_id = tokenizer.pad_token_id


2023-12-31 11:42:33 [INFO] CPU device is used.
2023-12-31 11:42:33 [INFO] Applying Weight Only Quantization.


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

2023-12-31 11:46:51 [INFO] Start auto tuning.
2023-12-31 11:46:51 [INFO] Quantize model without tuning!
2023-12-31 11:46:51 [INFO] Quantize the model with default configuration without evaluating the model.                To perform the tuning process, please either provide an eval_func or provide an                    eval_dataloader an eval_metric.
2023-12-31 11:46:51 [INFO] Adaptor has 5 recipes.
2023-12-31 11:46:51 [INFO] 0 recipes specified by user.
2023-12-31 11:46:51 [INFO] 3 recipes require future tuning.
2023-12-31 11:46:51 [INFO] *** Initialize auto tuning
2023-12-31 11:46:51 [INFO] {
2023-12-31 11:46:51 [INFO]     'PostTrainingQuantConfig': {
2023-12-31 11:46:51 [INFO]         'AccuracyCriterion': {
2023-12-31 11:46:51 [INFO]             'criterion': 'relative',
2023-12-31 11:46:51 [INFO]             'higher_is_better': True,
2023-12-31 11:46:51 [INFO]             'tolerable_loss': 0.01,
2023-12-31 11:46:51 [INFO]             'absolute': None,
2023-12-31 11:46:51 [INFO]     

The following cell only prints the architecture of the model.

In [ ]:
print(model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): QuantizedLinearQBits(in_features=4096, out_features=4096, bias=False)
          (k_proj): QuantizedLinearQBits(in_features=4096, out_features=1024, bias=False)
          (v_proj): QuantizedLinearQBits(in_features=4096, out_features=1024, bias=False)
          (o_proj): QuantizedLinearQBits(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): QuantizedLinearQBits(in_features=4096, out_features=14336, bias=False)
          (up_proj): QuantizedLinearQBits(in_features=4096, out_features=14336, bias=False)
          (down_proj): QuantizedLinearQBits(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Mistral

Define the configuration of LoRA.

In [ ]:
model.gradient_checkpointing_enable()
peft_config = LoraConfig(
    r=8,
    task_type=TaskType.CAUSAL_LM,
)

For this demonstration, since CPU fine-tuning is too slow on Colab, I trained for only 200 steps. I have also deactivated the evaluation.

In [ ]:
model = get_peft_model(model, peft_config)

In [ ]:
training_arguments = TrainingArguments(
        output_dir="./results_sft_cpu/",
        #evaluation_strategy="steps",
        #do_eval=True,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=2,
        per_device_eval_batch_size=4,
        log_level="debug",
        save_steps=20,
        logging_steps=10,
        learning_rate=1e-4,
        #eval_steps=10,
        max_steps=200,
        warmup_steps=20,
        lr_scheduler_type="linear",
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


Start training:

In [ ]:
trainer = SFTTrainer(
        model=model,
        train_dataset=dataset_train_sft,
        #\val_dataset=dataset_test_sft,
        dataset_text_field="text",
        max_seq_length=256,
        tokenizer=tokenizer,
        args=training_arguments,
)

trainer.train()

Map:   0%|          | 0/207865 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
Currently training with a batch size of: 2
***** Running training *****
  Num examples = 207,865
  Num Epochs = 1
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 2
  Total optimization steps = 200
  Number of trainable parameters = 3,407,872


Step,Training Loss
10,1.589800
20,1.473600
30,1.527100
40,1.457200
50,1.474200
60,1.417500
70,1.394800
80,1.332500
90,1.476500
100,1.324400


Saving model checkpoint to ./results_sft_cpu/checkpoint-20
tokenizer config file saved in ./results_sft_cpu/checkpoint-20/tokenizer_config.json
Special tokens file saved in ./results_sft_cpu/checkpoint-20/special_tokens_map.json
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
Saving model checkpoint to ./results_sft_cpu/checkpoint-40
tokenizer config file saved in ./results_sft_cpu/checkpoint-40/tokenizer_config.json
Special tokens file saved in ./results_sft_cpu/checkpoint-40/special_tokens_map.json
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: tor

Step,Training Loss
10,1.589800
20,1.473600
30,1.527100
40,1.457200
50,1.474200
60,1.417500
70,1.394800
80,1.332500
90,1.476500
100,1.324400


Saving model checkpoint to ./results_sft_cpu/checkpoint-200
tokenizer config file saved in ./results_sft_cpu/checkpoint-200/tokenizer_config.json
Special tokens file saved in ./results_sft_cpu/checkpoint-200/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=200, training_loss=1.416732668876648, metrics={'train_runtime': 70203.1913, 'train_samples_per_second': 0.011, 'train_steps_per_second': 0.003, 'total_flos': 5553146088652800.0, 'train_loss': 1.416732668876648, 'epoch': 0.0})